# 1. Import Libraries and datasets
---

## 1.1 Libraries 
---

In [ ]:
# 1. Import Libraries and Datasets
## 1.1 Libraries

import pandas as pd
import numpy as np
import timeit
import re

import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import nltk
nltk.download('punkt')

import textblob
from textblob import TextBlob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import torch
print(torch.__version__,' pytorch version')
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

!pip install transformers==2.6.0

1.6.0+cu101  pytorch version
     |████████████████████████████████| 542kB 5.0MB/s 
     |████████████████████████████████| 890kB 12.3MB/s 
     |████████████████████████████████| 133kB 24.3MB/s 
     |████████████████████████████████| 3.7MB 31.9MB/s 
     |████████████████████████████████| 1.1MB 50.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 6.7MB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=939c0588226b021ca5520a3d29e13cf0d45a06a800f77127512533a4b5652877
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import transformers
print(transformers.__version__,' make sure transformers version is 2.6.0')
from transformers import GPT2Tokenizer

2.6.0  make sure transformers version is 2.6.0


## 1.2 Tokenizer and datasets
---

In [ ]:
## 1.2 Tokenizer and Datasets

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
special_tokens = {'bos_token':'<|startoftext|>','eos_token':'<|endoftext|>','pad_token':'<pad>','additional_special_tokens':['<|keyword|>','<|summarize|>']}
tokenizer.add_special_tokens(special_tokens)
print(len(tokenizer), 'total length of vocab')
print(tokenizer.bos_token_id, 'bos_token')
print(tokenizer.eos_token_id, 'eos_token')
print(tokenizer.pad_token_id, 'pad_token')  #token for <pad>, len of all tokens in the tokenizer
print(tokenizer.additional_special_tokens_ids[0], 'keyword_token') #token for <|keyword|>
print(tokenizer.additional_special_tokens_ids[1], 'summary_token') #token for <|summarize|>


50261 total length of vocab
50257 bos_token
50256 eos_token
50258 pad_token
50259 keyword_token
50260 summary_token


In [ ]:
df_train = pd.read_csv('train_last_edt.csv',index_col=0)
df_test  = pd.read_csv('test_last_edt.csv',index_col=0)

In [ ]:
#df_train.head()

# 2. Generate keywords
---

In [ ]:
# 2. Generate Keywords

def get_keywords(row):
    some_text = row['for_keyword']
    lowered = some_text.lower()
    tokens = nltk.tokenize.word_tokenize(lowered)
    keywords = [keyword for keyword in tokens if keyword.isalpha() and not keyword in stop_words]
    keywords_string = ','.join(keywords)
    return keywords_string

In [ ]:
def pre_process_ogi(df):
  deneme = df
  deneme['for_keyword'] = deneme['headlines'] + deneme['ctext']
  deneme['keywords'] = deneme.apply(get_keywords, axis=1)
  deneme['keyword_POS'] = deneme["keywords"].apply(lambda x: TextBlob(x).words)
  deneme["keyword_POS_str"] = deneme.keyword_POS.apply(', '.join)
  deneme['keyword_POS_str_nocomma'] = deneme.keyword_POS.apply(' '.join)
  
  return deneme

In [ ]:
train_process = pre_process_ogi(df_train)
test_process = pre_process_ogi(df_test)

In [ ]:
#test_process.head()

# 3. Preprocess for gpt training
---

## 3.1 Necessary Functions
---

In [ ]:
# 3. Pre-Process for GPT Training

## 3.1 Requirement Mini Functions

def load_words(df, num,with_title = False):
    """import dataframe with number of what sample to choose,
    return a keyword (together with title or not) as strings
    and abstract for summarization.
    and 3 distractors. all as a tuple of 5 strings"""
    arr_distract = np.random.randint(len(df), size=3)
    keyword = df['keyword_POS_str'][num]
    if with_title:
        title = df['headlines'][num]
        keyword = title + keyword
    abstract = df['ctext'][num]
    part1 = df['ctext'][arr_distract[0]]
    part2 = df['ctext'][arr_distract[1]]
    part3 = df['ctext'][arr_distract[2]]

    return (keyword,abstract,part1,part2,part3)

def tag_pull_abstract(df, list_POS):
    """ return list of keyword list
    input: pandas dataframe
                    list of part of speech tag (in order to generate keyword)
    ourput: List(List(keyword string))"""
    list_tokenized = df['ctext'].apply(
        lambda x: nltk.pos_tag(nltk.word_tokenize(x))).values
    list_answer = [[item[0] for item in row if item[1] in list_POS]
                   for row in list_tokenized]
    #list_answer = list(map(lambda x: ' '.join(x), list_answer))
    return list_answer


def write_input_ids(word_batch,max_len=1024):
    """return list of input tokens"""
    keyword, abstract, dis1,dis2,dis3 = word_batch

    input_true = tokenizer.encode('<|startoftext|> ' + keyword + ' <|summarize|> '+ abstract + ' <|endoftext|>',max_length = tokenizer.max_len)
    input_dis1 = tokenizer.encode('<|startoftext|> ' + keyword + ' <|summarize|> '+ dis1 + ' <|endoftext|>',max_length = tokenizer.max_len)
    input_dis2 = tokenizer.encode('<|startoftext|> ' + keyword + ' <|summarize|> '+ dis2 + ' <|endoftext|>',max_length = tokenizer.max_len)
    input_dis3 = tokenizer.encode('<|startoftext|> ' + keyword + ' <|summarize|> '+ dis3 + ' <|endoftext|>',max_length = tokenizer.max_len)
  
    if max_len == None:
        max_len = max(len(input_true),len(input_dis1),len(input_dis2),len(input_dis3))
    for i in [input_true,input_dis1,input_dis2,input_dis3]:
        while len(i) < max_len:
            i.append(tokenizer.pad_token_id)
    list_input_token = [input_true,input_dis1,input_dis2,input_dis3]
    return list_input_token


def write_token_type_labels(list_input_ids,max_len=1024):
    list_segment = []
    for item in list_input_ids:
        try:
            item.index(tokenizer.eos_token_id)
        except:
            item[-1] = tokenizer.eos_token_id
        num_seg_a = item.index(tokenizer.additional_special_tokens_ids[1]) + 1
        end_index = item.index(tokenizer.eos_token_id)
        num_seg_b = end_index - num_seg_a + 1
        num_pad = max_len - end_index - 1
        segment_ids = [tokenizer.additional_special_tokens_ids[0]]*num_seg_a + [tokenizer.additional_special_tokens_ids[1]]*num_seg_b + [tokenizer.pad_token_id]*num_pad
        list_segment.append(segment_ids)
    return list_segment


def write_lm_labels(list_input_ids,list_type_labels):
    list_lm_label = []
    is_true_label = True
    for input_tokens,segments in zip(list_input_ids,list_type_labels):
        if is_true_label:
            is_true_label = False
            temp_list = []
            for token,segment in zip(input_tokens,segments):
                if segment == tokenizer.additional_special_tokens_ids[1]:
                    temp_list.append(token)
                else:
                    temp_list.append(-100)
            list_lm_label.append(temp_list)
        else:
            temp_list = [-100]*len(input_tokens)
            list_lm_label.append(temp_list)
    return list_lm_label


def execute_tokenization(word_tuple):
    list_input_ids = write_input_ids(word_tuple)
    list_type_labels = write_token_type_ids(list_input_ids)
    list_last_tokens = write_last_token(list_input_ids)
    list_lm_label = write_lm_labels(list_input_ids, list_type_labels)
    list_mc_labels = write_mc_labels()
    np_tuple = shuffle_batch(list_input_ids, list_type_labels,
                             list_last_tokens, list_lm_label, list_mc_labels)
    tensor_tuple = write_torch_tensor(np_tuple)
    return tensor_tuple

def write_last_token(list_input_ids):
    list_mc_token = []
    for item in list_input_ids:
        list_mc_token.append(item.index(tokenizer.eos_token_id))
    return list_mc_token

def write_mc_label():
    return [1,0,0,0]

def shuffle_batch(list_input_ids,list_type_labels,list_last_tokens,list_lm_labels,list_mc_labels):
    array_input_token = np.array(list_input_ids)
    array_segment = np.array(list_type_labels)
    array_mc_token = np.array(list_last_tokens)
    array_lm_label = np.array(list_lm_labels)
    array_mc_label = np.array(list_mc_labels)

    randomize = np.arange(4)
    np.random.shuffle(randomize)

    array_input_token = array_input_token[randomize]
    array_segment = array_segment[randomize]
    array_mc_token = array_mc_token[randomize]
    array_lm_label = array_lm_label[randomize]
    array_mc_label = array_mc_label[randomize]

    return (array_input_token,array_segment,array_mc_token,array_lm_label,array_mc_label)

def write_torch_tensor(np_batch):
    torch_input_token = torch.tensor(np_batch[0], dtype=torch.long).unsqueeze(0)
    torch_segment = torch.tensor(np_batch[1],dtype=torch.long).unsqueeze(0)
    torch_mc_token = torch.tensor(np_batch[2],dtype=torch.long).unsqueeze(0)
    torch_lm_label = torch.tensor(np_batch[3],dtype=torch.long).unsqueeze(0)
    torch_mc_label = torch.tensor([np.argmax(np_batch[4])],dtype=torch.long).unsqueeze(0)
    return (torch_input_token,torch_segment,torch_mc_token,torch_lm_label,torch_mc_label)

### 3.1.1 Lets see an example
---

In [ ]:
### 3.1.1 Example Preview
key_batch = load_words(train_process,123)

In [ ]:
key_batch

('elderly, kills, self, pledges, donate, organs, suicide, notea, south, west, delhi, dwarka, sector, allegedly, committed, suicide, tuesday, note, requested, organs, donated, india, institute, medical, thanked, doctors, police, officials, suicide, note, deceased, identified, prahlad, narayan, mishra, said, tired, life, man, committed, suicide, apartment, using, cloth, hang, ceiling, fan, senior, police, official, said, wife, son, used, live, rushed, local, hospital, declared, dead, deceased, retired, civil, engineer, worked, advisor, defence, research, development, organization, drdo, hotels, survived, wife, three, sons, two, united, states, america',
 'A 66-year-old from South West Delhi?s Dwarka Sector 22 allegedly committed suicide on Tuesday and in a note requested his organs be donated to All India Institute of Medical Sciences.He thanked doctors and police officials in advance.In his suicide note, the deceased, who has been identified as Prahlad Narayan Mishra, said ?he is tired 

## 3.2 Masking
---

In [ ]:
## 3.2 Masking

def execute_all_mini_function(df):
  exist_temp_tensor = False
  exist_big_tensor = False
  start = timeit.default_timer()
  for num in range(len(df)):
    # print(num)  # I used this to find lines with errors
    word_tuple = load_words(df, num)
    if type(word_tuple[0]) != str or type(word_tuple[1]) != str:
        continue
    
    list_input_ids   = write_input_ids(word_tuple)
    list_type_labels = write_token_type_labels(list_input_ids)
    list_lm_labels   = write_lm_labels(list_input_ids,list_type_labels)
    list_last_tokens = write_last_token(list_input_ids)
    list_mc_labels   = write_mc_label()

    np_tuple = shuffle_batch(list_input_ids,list_type_labels,list_last_tokens,list_lm_labels,list_mc_labels)
    tensor_tuple = write_torch_tensor(np_tuple)
    
    if not exist_temp_tensor:
      temp_0 = tensor_tuple[0]
      temp_1 = tensor_tuple[1]
      temp_2 = tensor_tuple[2]
      temp_3 = tensor_tuple[3]
      temp_4 = tensor_tuple[4]
      exist_temp_tensor = True
    elif exist_temp_tensor:
      temp_0 = torch.cat((temp_0,tensor_tuple[0]),0)
      temp_1 = torch.cat((temp_1,tensor_tuple[1]),0)
      temp_2 = torch.cat((temp_2,tensor_tuple[2]),0)
      temp_3 = torch.cat((temp_3,tensor_tuple[3]),0)
      temp_4 = torch.cat((temp_4,tensor_tuple[4]),0)

    if num % 1000 == 0:
      if not exist_big_tensor:
        big_first_tensor = temp_0
        big_second_tensor = temp_1
        big_third_tensor = temp_2
        big_fourth_tensor = temp_3
        big_fifth_tensor = temp_4
        exist_temp_tensor = False
        exist_big_tensor = True
        del temp_0,temp_1,temp_2,temp_3,temp_4
      else:
        big_first_tensor = torch.cat((big_first_tensor,temp_0),0)
        big_second_tensor = torch.cat((big_second_tensor,temp_1),0)
        big_third_tensor = torch.cat((big_third_tensor,temp_2),0)
        big_fourth_tensor = torch.cat((big_fourth_tensor,temp_3),0)
        big_fifth_tensor = torch.cat((big_fifth_tensor,temp_4),0)
        exist_temp_tensor = False
        del temp_0,temp_1,temp_2,temp_3,temp_4
      
      stop = timeit.default_timer()
      print('iterations ',num,' takes ', stop - start,' sec')
      start = timeit.default_timer()
  
  big_first_tensor = torch.cat((big_first_tensor,temp_0),0)
  big_second_tensor = torch.cat((big_second_tensor,temp_1),0)
  big_third_tensor = torch.cat((big_third_tensor,temp_2),0)
  big_fourth_tensor = torch.cat((big_fourth_tensor,temp_3),0)
  big_fifth_tensor = torch.cat((big_fifth_tensor,temp_4),0)
  return big_first_tensor, big_second_tensor, big_third_tensor,big_fourth_tensor,big_fifth_tensor

In [ ]:
tensor_1,tensor_2,tensor_3,tensor_4,tensor_5 = execute_all_mini_function(train_process)

iterations  0  takes  0.03386694599976181  sec
iterations  1000  takes  28.75640174399996  sec
iterations  2000  takes  28.58379504000004  sec
iterations  3000  takes  28.66317658400021  sec


In [ ]:
# create a tensor dataset object
tensor_dataset = TensorDataset(tensor_1,tensor_2,tensor_3,tensor_4,tensor_5)

In [ ]:
# save the tensor object to load later when training
torch.save(tensor_dataset, 'torch_file_from_train.pt')

In [ ]:
tensor_1,tensor_2,tensor_3,tensor_4,tensor_5 = execute_all_mini_function(test_process)

iterations  0  takes  0.01976048000051378  sec
iterations  1000  takes  28.794872772000417  sec


In [ ]:
# create a tensor dataset object
tensor_dataset = TensorDataset(tensor_1,tensor_2,tensor_3,tensor_4,tensor_5)

In [ ]:
# save the tensor object to load later when training
torch.save(tensor_dataset, 'torch_file_from_test.pt')

### 3.2.1 See an example
---

In [ ]:
### 3.2.1 Example
item = 123
print(tensor_1[item])
print(tensor_2[item])
print(tensor_3[item])
print(tensor_4[item])
print(tensor_5[item])

tensor([[50257,   264,   380,  ...,   345,  1600, 50256],
        [50257,   264,   380,  ..., 50258, 50258, 50258],
        [50257,   264,   380,  ...,    67,  2265, 50256],
        [50257,   264,   380,  ...,   319,  1171, 50256]])
tensor([[50259, 50259, 50259,  ..., 50260, 50260, 50260],
        [50259, 50259, 50259,  ..., 50258, 50258, 50258],
        [50259, 50259, 50259,  ..., 50260, 50260, 50260],
        [50259, 50259, 50259,  ..., 50260, 50260, 50260]])
tensor([1023,  828, 1023, 1023])
tensor([[ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100],
        [ -100,  -100,  -100,  ...,    67,  2265, 50256],
        [ -100,  -100,  -100,  ...,  -100,  -100,  -100]])
tensor([2])


In [ ]:
print('{:>2}{:>10}{:>10}{:>10}{:>10}{:>20}{:>10}{:>20}{:>10}'.format('count','input','decoded input','input','decoded input','input','decoded input','input','decoded input'))
count = 0
for i,j,k,m in zip(tensor_1[item][1],tensor_1[item][2],tensor_2[item][2],tensor_4[item][2]):
  i = int(i)
  j = int(j)
  k = int(k)
  m = int(m)
  if i == -100:
    decode_i = 'masked'
  else:
    decode_i = tokenizer.decode(i)
  if j == -100:
    decode_j = 'masked'
  else:
    decode_j = tokenizer.decode(j)
  if k == -100:
    decode_k = 'masked'
  else:
    decode_k = tokenizer.decode(k)
  if m == -100:
    decode_m = 'masked'
  else:
    decode_m = tokenizer.decode(m)
  #print(i,j)
  print('{:>2}{:>10}{:>10}{:>10}{:>10}{:>20}{:>10}{:>20}{:>10}'.format(count,i,decode_i,j,decode_j,k,decode_k,m,decode_m))
  count += 1

count     inputdecoded input     inputdecoded input               inputdecoded input               inputdecoded input
 0     50257<|startoftext|>     50257<|startoftext|>               50259<|keyword|>                -100    masked
 1       264         s       264         s               50259<|keyword|>                -100    masked
 2       380        ri       380        ri               50259<|keyword|>                -100    masked
 3        11         ,        11         ,               50259<|keyword|>                -100    masked
 4       300         l       300         l               50259<|keyword|>                -100    masked
 5       962       ank       962       ank               50259<|keyword|>                -100    masked
 6       272        an       272        an               50259<|keyword|>                -100    masked
 7        11         ,        11         ,               50259<|keyword|>                -100    masked
 8      5701    sports      5701    spor